In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')


In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train['Cabin'].unique()

In [ ]:
train['Survived'].value_counts()

In [ ]:
train['Pclass'].value_counts().sort_index()

In [ ]:
train['Sex'].value_counts().sort_index()

In [ ]:
train['Embarked'].value_counts().sort_index()

In [ ]:
train.isnull().any()

In [ ]:
train.isnull().mean()

In [ ]:
train['Embarked'] = train['Embarked'].fillna('S')

In [ ]:
train['Age'].fillna(train['Age'].median(), inplace=True)

In [ ]:
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand = False)
train['Title'].unique().tolist()

In [ ]:
train['Title'] = train['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
train['Title'] = train['Title'].replace(['Jonkheer', 'Master'], 'Master')
train['Title'] = train['Title'].replace(['Don', 'Sir', 'the Countess', 'Lady', 'Dona'], 'Royalty')
train['Title'] = train['Title'].replace(['Mme', 'Ms', 'Mrs'], 'Mrs')
train['Title'] = train['Title'].replace(['Mlle', 'Miss'], 'Miss')

train['Title'].value_counts()

In [ ]:
y = train['Survived']

In [ ]:
bins = [0, 18, 25, 35, 60, 100]
group_names = ['Baby', 'Youth', 'YoungAdult', 'MiddleAged', 'Senior']
train['AgeGroup'] = pd.cut(train['Age'], bins, labels=group_names)
train['AgeGroup'].value_counts()

In [ ]:
train[train['AgeGroup'] == 'Baby']

In [ ]:
import seaborn as sns
import matplotlib.pylab as plt

In [ ]:
train.hist(figsize=(14,14), bins=20)
plt.show()

In [ ]:
sns.barplot(x="Sex", y="Survived", data=train)

In [ ]:
sns.barplot(x="Pclass", y="Survived", hue ='Sex', data=train)

In [ ]:
plt.subplots(figsize=(10, 10))
sns.heatmap(train.corr(), annot=True, linewidths=.2)

In [ ]:
sns.barplot(x="AgeGroup", y="Survived", hue ='Pclass', data=train)

In [ ]:
sns.barplot(x="SibSp", y="Survived",hue ='Pclass', data=train)

In [ ]:
train.drop(['Name', 'Ticket', 'SibSp', 'Parch', 'Cabin', 'AgeGroup'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

for col in ['Sex', 'Embarked', 'Title']:
    train[col] = label.fit_transform(train[col])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state = 5, stratify = y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 10, 
                                  n_estimators = 26,
                                  max_depth = 6)
model.fit(x_train, y_train)
model.predict(x_valid)

In [ ]:
from lightgbm import LGBMClassifier
# 31개 기본임, 잎사귀가 많아질수록 성능이 좋아짐, 너무 많아지면 과적합이되서 성능이 안좋아짐.
# LGBMClassifier은 트리모델임, 카테고리형은 다르다고 생각하고 분리함
#max_bin = 465, max_delta_step = 0.9, learning_rate=0.4, num_leaves = 42, n_estimators=100,
#lgb = LGBMClassifier(n_estimators = 50, num_leaves = 28)
lgb = LGBMClassifier(max_bin = 465, max_delta_step = 0.9, learning_rate=0.4, num_leaves = 42, n_estimators=100)

## 테스트용 코드
    #early_stopping_rounds 오버피팅방지, 성능이 안좋아지면 자동으로 멈춤
lgb.fit(x_train, y_train, eval_set = (x_valid, y_valid), early_stopping_rounds= 20 ) 

## 실제 제출시 코드
#lgb.fit(train, y)
#preds = lgb.predict(test)
#분류문제는 각 분류에 대한 확률값 제출 predict_proba
#preds = lgb.predict_proba(test)
#submission = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv')